# Filtro en tiempo real al estilo de Snapchat con OpenCV

En esta práctica vas a crear un filtro en tiempo real que añade orejas y nariz de perrito sobre tu cara, usando Python y OpenCV. Aprenderás a:
- Cargar y usar un clasificador Haar Cascade para detección de rostros.
- Leer un gráfico PNG con canal alfa (transparencia) como overlay.
- Redimensionar y posicionar el overlay según la detección de la cara.
- Mezclar (blend) el overlay PNG sobre cada frame de vídeo de la cámara.

Como lo hemos hecho a lo largo del curso, comenzamos con importar las librerías necesarias.

In [1]:
import cv2
import numpy as np

Para esta práctica usaremos una herramienta para detectar rostros llamado `Haar Cascade`

Si por alguna extraña razón no les abre el clasificador Haar, pueden buscarlo y descargarlo en este [enlace](https://github.com/npinto/opencv/blob/master/data/haarcascades/haarcascade_frontalface_default.xml).
Solo deben asegurarse de que el archivo `.xml` se encuentre en el mismo directorio de trabajo que este cuaderno.

In [2]:
# Cargamos el clasificador Haar Cascade para detección de rostros
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [3]:
# Cargamos el overlay de orejas y nariz de perro en formato png
# Si quieren usar otro overlay asegurense de que tenga un canal alpha (el png sea transparente, sin fondo)
dog_filter_png = cv2.imread('dog_face_full.png', cv2.IMREAD_UNCHANGED)

Ahora vamos a crear una función para superponer el *overlay* sobre la imagen de nuestro rostro.

La función auxiliar tomará un frame BGR y el *overlay* en RGBA, mezclará estas dos imágenes en la posición deseada usando el canal *alpha* de la imagen.

In [4]:
# Función para overlay con canal alfa

def overlay_png_alpha(frame, overlay_png, x, y, w, h):
    """
    Superpone overlay_png (con canal alfa) en frame BGR.
    - frame: imagen destino (BGR)
    - overlay_png: imagen fuente (BGRA)
    - (x, y): esquina superior izquierda donde colocar el overlay
    - (w, h): tamaño al que redimensionar el overlay
    """
    # Redimensionar overlay al tamaño deseado
    overlay_resized = cv2.resize(overlay_png, (w, h), interpolation=cv2.INTER_AREA)
    
    # Separar canales BGR y alpha
    b, g, r, a = cv2.split(overlay_resized)
    overlay_rgb = cv2.merge((b, g, r))
    mask = cv2.cvtColor(a, cv2.COLOR_GRAY2BGR) / 255.0  # Normalizar alpha [0,1]
    
    # Región de interés en el frame
    roi = frame[y:y+h, x:x+w].astype(float) / 255.0
    
    # Mezclar: out = alpha*overlay + (1-alpha)*fondo
    blended = (mask * overlay_rgb.astype(float) / 255.0) + ((1 - mask) * roi)
    frame[y:y+h, x:x+w] = np.uint8(blended * 255)


Ya con nuestra función definida, ahora vamos a repetir los pasos para inicializar la cámara y leemos las imágenes en un bucle infinito, es decir, la captura de video con OpenCV. Además de capturar video, ahora estaremos detectando rostros y aparte aplicando el filtro sobre la imagen.

Nota: el desempeño del filtro dependerá enteramente de las capacidades de su computadora ya que el procesamiento del video es una tarea bastante compleja, computacionalmente hablando.

In [ ]:
# Capturar el video y aplicar el filtro
cap = cv2.VideoCapture(0) # Cambiar el puerto de la cámara en caso de ser necesario

if not cap.isOpened():
    raise IOError("No se puede acceder a la cámara")

# Bucle infinito
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convertir a escala de grises para realizar una detección más rápida
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detectar todos los rostros dentro del frame
    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor = 1.1,
        minNeighbors = 5,
        minSize = (100,100)
    )

    # Para cada rostro detectado, colocar el overlay
    for (x,y,w,h) in faces:
        # Pueden ajustar la posición y tamaño de las orejas si es necesario
        ears_w = int(w * 1.2) # Ancho de las orejas
        ears_h = int(h * 0.6) # Alto de las orejas
        ears_x = x - (ears_w - w) // 2 # Posición en el eje x de las orejas
        ears_y = y - int(h * 0.5) # Posición en el eje y de las orejas

        # Colocar las orejas
        overlay_png_alpha(frame, dog_filter_png, ears_x, ears_y, ears_w, ears_h)

        # # Por si quieren colocar la nariz centrada en la parte inferior de la cara
        # nose_w = int(w * 0.4)
        # nose_h = int(h * 0.3)
        # nose_x = x + (w - nose_w) // 2
        # nose_y = y + int(h * 0.55)
        # overlay_png_alpha(frame, dog_filter_png, nose_x, nose_y, nose_w, nose_h)

    # Mostrar el resultado en pantalla
    cv2.imshow('Filtro de perrito', frame)

    # Salir si presionamos la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar los recursos
cap.release()
cv2.destroyAllWindows()

ValueError: operands could not be broadcast together with shapes (111,222,3) (0,222,3) 

: 

# Comentarios y extensiones posibles

- Ajustar los factores de escala (`ears_w`, `ears_h`, `nose_w`, `nose_h`) para distintos tamaños de cara.
- Usar un detector basado en landmarks (dlib, mediapipe) para situar con precisión ojos, nariz, boca.
- Añadir animaciones: parpadeo de ojos, movimiento de lengua.
- Reemplazar el PNG por otros filtros: gafas, sombreros, máscaras, etc.
- Experimentar con blending más suave (gaussian blur en la máscara alfa).
